In [4]:
import os
import cv2
import time
import numpy as np

from ultralytics import YOLO
from scipy.spatial.distance import euclidean
from deep_sort_realtime.deepsort_tracker import DeepSort

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
def assign_stable_id(track_id, x_center, y_center, stable_id_map, last_positions, next_stable_id, max_stable_ids):
    if track_id not in stable_id_map:
        if next_stable_id <= max_stable_ids:
            stable_id_map[track_id] = next_stable_id
            next_stable_id += 1
        else:
            # Assign to closest stable ID based on distance
            dists = {
                sid: np.linalg.norm(
                    np.array([x_center, y_center]) - np.array(last_positions.get(sid, [np.inf, np.inf]))
                ) for sid in range(1, max_stable_ids + 1)
            }
            stable_id_map[track_id] = min(dists, key=dists.get)
    return stable_id_map[track_id], next_stable_id

In [6]:
def check_battle_start(last_positions, current_positions, speed_threshold=2.0):
    """
    Checks if the battle started:
    - Both Beyblades (stable_id 1 and 2) detected
    - Both moving faster than speed_threshold (pixels/frame)
    """
    if 1 in last_positions and 2 in last_positions:
        return True
    
    elif 1 not in last_positions or 2 not in last_positions:
        return False  # Both not detected yet

    elif 1 not in current_positions or 2 not in current_positions:
        return False  # Both not detected in current frame

    # speed_1 = np.linalg.norm(np.array(current_positions[1]) - np.array(last_positions[1]))
    # speed_2 = np.linalg.norm(np.array(current_positions[2]) - np.array(last_positions[2]))

    # return speed_1 > speed_threshold and speed_2 > speed_threshold

In [7]:
def angle_between(v1, v2):
    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-8)
    return np.arccos(np.clip(cos_theta, -1.0, 1.0))

def midpoint(p1, p2):
    return ((p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2)

In [8]:
def compute_iou(boxA, boxB):
    # box = (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

In [9]:
model = YOLO('./runs/detect/train7/weights/best.pt')  # Load the best model from training
stable_id_map = {}  # tracker_id -> stable_id
last_positions = {}  # stable_id -> (x_center, y_center)
next_stable_id = 1
max_stable_ids = 2
battle_started = False
detected_broken_beyblades = False
check_first_beyblade = True
check_battle_ends = True
flag_for_stopping_collision = True
total_collision = 0
start_time = 0
stable_id = None
prev_c1 = None
prev_c2 = None
stop_frame_count_1 = 0
stop_frame_count_2 = 0
c1 = None
c2 = None

cap = cv2.VideoCapture('./source video/beyblade battle 1 clean.mov')

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi
out = cv2.VideoWriter('./beyblade battle 1 clean tracking.mov', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame, 
        conf=0.5, 
        tracker='botsort.yaml',
        persist=True,
        verbose=False)

    boxes = results[0].boxes  # YOLO detections with BoT-SORT IDs
    if len(boxes) > 2:
        detected_broken_beyblades = True

    current_positions = {}
    beyblade_1_positions = {}
    beyblade_2_positions = {}

    for box in boxes:
        if box.id is None:
            continue

        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        
        # Assign stable ID
        stable_id, next_stable_id = assign_stable_id(
                track_id, x_center, y_center,
                stable_id_map, last_positions, next_stable_id, max_stable_ids
            )
        
        last_positions[stable_id] = (x_center, y_center)

        # To save the coordinates bbox for each object track per frame
        if stable_id == 1 and len(boxes) < 3:
            beyblade_1_positions['x1'] = x1
            beyblade_1_positions['y1'] = y1
            beyblade_1_positions['x2'] = x2
            beyblade_1_positions['y2'] = y2
            beyblade_1_positions['box'] = box
        elif stable_id == 2 and len(boxes) < 3:
            beyblade_2_positions['x1'] = x1
            beyblade_2_positions['y1'] = y1
            beyblade_2_positions['x2'] = x2
            beyblade_2_positions['y2'] = y2
            beyblade_2_positions['box'] = box

        # Draw bbox and stable ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Beyblade {stable_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
        

    # Check First Beyblade launch time
    if check_first_beyblade and stable_id == 1:
        print("First Beyblade Launch!")
        
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # ← current frame number
        current_time = current_frame / fps                     # ← time in seconds

        print(f"Frame: {current_frame}, Time: {current_time:.2f}s")

        check_first_beyblade = False

    # Check if battle has started
    if len(boxes.cls) > 0:
        current_positions[stable_id] = (x_center, y_center)
    
    if not battle_started and check_battle_start(last_positions, current_positions):
        battle_started = True
        print("Second Beyblade Launch and Battle started!")
        
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # ← current frame number
        current_time = current_frame / fps                     # ← time in seconds

        print(f"Frame: {current_frame}, Time: {current_time:.2f}s")

    prev_c1 = c1
    prev_c2 = c2

    # Check First Collision
    if len(boxes) > 1:
        try:
            c1 = midpoint((beyblade_1_positions['x1'], beyblade_1_positions['y1']),
                        (beyblade_1_positions['x2'], beyblade_1_positions['y2']))
            c2 = midpoint((beyblade_2_positions['x1'], beyblade_2_positions['y1']),
                        (beyblade_2_positions['x2'], beyblade_2_positions['y2']))
            
            distance = euclidean(c1, c2)
            bbox_width = beyblade_2_positions['x2'] - beyblade_2_positions['x2'] 

            distance_thresh = bbox_width * 0.6  # more dynamic
            
            boxA = list(map(int, beyblade_1_positions['box'].xyxy[0]))
            boxB = list(map(int, beyblade_2_positions['box'].xyxy[0]))
            iou = compute_iou(boxA, boxB)
        
            if distance < distance_thresh or iou > 0.05 and flag_for_stopping_collision:
                print(f"💥 Collision Happened!")
                current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # ← current frame number
                current_time = current_frame / fps                     # ← time in seconds

                print(f"Frame: {current_frame}, Time: {current_time:.2f}s")

                total_collision += 1
                
        except:
            pass

    if detected_broken_beyblades and check_battle_ends:
        print('Battle ends!')

        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # ← current frame number
        current_time = current_frame / fps                     # ← time in seconds

        print(f"Frame: {current_frame}, Time: {current_time:.2f}s")
        
        check_battle_ends = False
        flag_for_stopping_collision = False


    if detected_broken_beyblades:
    # Winner Declaration
    # Calculate velocity
        v1 = np.linalg.norm(np.array(c1) - np.array(prev_c1))
        v2 = np.linalg.norm(np.array(c2) - np.array(prev_c2))

        # Define stop threshold
        motion_thresh = 2  # pixels per frame
        stop_frame_count_1 += 1 if v1 < motion_thresh else 0
        stop_frame_count_2 += 1 if v2 < motion_thresh else 0

        # If one is stopped for N frames, declare the other the winner
        if stop_frame_count_1 > 30 and stop_frame_count_2 <= 30:
            print("🏆 Beyblade 2 wins!")
        elif stop_frame_count_2 > 30 and stop_frame_count_1 <= 30:
            print("🏆 Beyblade 1 wins!")


    if battle_started:
        cv2.putText(frame, 'Battle Started!', (50, 50), 
            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
    
    if detected_broken_beyblades:
        cv2.putText(frame, 'DETECTED BROKEN BEYBLADES', (250, 250), 
            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    out.write(frame)

cap.release()
out.release()

First Beyblade Launch!
Frame: 50, Time: 1.67s
Second Beyblade Launch and Battle started!
Frame: 113, Time: 3.77s
💥 Collision Happened!
Frame: 117, Time: 3.90s
💥 Collision Happened!
Frame: 118, Time: 3.93s
💥 Collision Happened!
Frame: 119, Time: 3.97s
💥 Collision Happened!
Frame: 139, Time: 4.63s
💥 Collision Happened!
Frame: 140, Time: 4.67s
💥 Collision Happened!
Frame: 141, Time: 4.70s
💥 Collision Happened!
Frame: 142, Time: 4.73s
💥 Collision Happened!
Frame: 143, Time: 4.77s
💥 Collision Happened!
Frame: 152, Time: 5.07s
💥 Collision Happened!
Frame: 153, Time: 5.10s
💥 Collision Happened!
Frame: 154, Time: 5.13s
💥 Collision Happened!
Frame: 200, Time: 6.67s
💥 Collision Happened!
Frame: 201, Time: 6.70s
💥 Collision Happened!
Frame: 217, Time: 7.23s
💥 Collision Happened!
Frame: 218, Time: 7.27s
💥 Collision Happened!
Frame: 219, Time: 7.30s
💥 Collision Happened!
Frame: 221, Time: 7.37s
💥 Collision Happened!
Frame: 259, Time: 8.63s
💥 Collision Happened!
Frame: 272, Time: 9.07s
💥 Collision H

# Test Package

In [1]:
import beyblade_tracker as bt

In [2]:
analyzer = bt.BeybladeBattleAnalyzer(
    model_path='./runs/detect/train7/weights/best.pt',
    video_path='./source video/beyblade battle 1 clean.mov',
    output_path='./beyblade battle 1 clean tracking.mov'
)
analyzer.run_analysis()

🚀 First Beyblade Launched at 1.67s
⚔️ Battle Started at 3.77s
💥 Collision Detected at 3.90s
💥 Collision Detected at 3.93s
💥 Collision Detected at 3.97s
💥 Collision Detected at 4.63s
💥 Collision Detected at 4.67s
💥 Collision Detected at 4.70s
💥 Collision Detected at 4.73s
💥 Collision Detected at 4.77s
💥 Collision Detected at 5.07s
💥 Collision Detected at 5.10s
💥 Collision Detected at 5.13s
💥 Collision Detected at 6.67s
💥 Collision Detected at 6.70s
💥 Collision Detected at 7.23s
💥 Collision Detected at 7.27s
💥 Collision Detected at 7.30s
💥 Collision Detected at 7.37s
💥 Collision Detected at 8.63s
💥 Collision Detected at 9.07s
💥 Collision Detected at 9.10s
💥 Collision Detected at 9.13s
🛑 Battle Ended at 9.40s


In [1]:
import ultralytics
import cv2
import numpy as np
import pandas as pd
import torch

In [2]:
print("ultralytics:", ultralytics.__version__)
print("opencv-python:", cv2.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("torch:", torch.__version__)

ultralytics: 8.3.146
opencv-python: 4.11.0
numpy: 2.2.6
pandas: 2.2.3
torch: 2.7.0+cu126
